In [1]:
import torch
from PIL import Image
import torchvision.transforms as T
import os
import numpy

In [2]:
model = torch.load("./skin-model-pokemon.pt", map_location=torch.device('cpu'))

C:\Users\user\AppData\Local\Temp\ipykernel_25532\1414262581.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("./skin-model-pokemon.pt", map_location=to

In [3]:
model.eval()

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(
 

In [4]:
data_dir = './datasets'
classes = os.listdir(data_dir)
classes

['acanthosis-nigricans',
 'acne',
 'acne-scars',
 'alopecia-areata',
 'dry',
 'melasma',
 'oily',
 'vitiligo',
 'warts']

In [5]:
# Test on an image
img = Image.open("./datasets/acne/acne_2.jpg").convert("RGB")

In [6]:
def get_transforms():
    transform = []
    transform.append(T.Resize((512, 512)))
    transform.append(T.ToTensor())
    return T.Compose(transform)

In [7]:
tr = get_transforms()

In [8]:
img_tensor = tr(img)

In [9]:
img_tensor

tensor([[[0.6902, 0.6392, 0.5922,  ..., 0.0706, 0.0667, 0.0549],
         [0.6745, 0.6549, 0.6078,  ..., 0.0824, 0.0824, 0.0706],
         [0.6627, 0.6706, 0.6314,  ..., 0.1020, 0.1020, 0.0902],
         ...,
         [0.5922, 0.5608, 0.4902,  ..., 0.6510, 0.6510, 0.6510],
         [0.5922, 0.5490, 0.4784,  ..., 0.6471, 0.6471, 0.6471],
         [0.6392, 0.5765, 0.4980,  ..., 0.6471, 0.6471, 0.6471]],

        [[0.5059, 0.4549, 0.4078,  ..., 0.0588, 0.0549, 0.0431],
         [0.4902, 0.4706, 0.4235,  ..., 0.0706, 0.0706, 0.0588],
         [0.4745, 0.4824, 0.4431,  ..., 0.0902, 0.0902, 0.0784],
         ...,
         [0.6196, 0.5765, 0.4902,  ..., 0.4706, 0.4706, 0.4706],
         [0.6157, 0.5647, 0.4784,  ..., 0.4667, 0.4667, 0.4667],
         [0.6627, 0.5961, 0.4980,  ..., 0.4667, 0.4667, 0.4667]],

        [[0.4275, 0.3765, 0.3294,  ..., 0.0863, 0.0824, 0.0706],
         [0.4118, 0.3922, 0.3451,  ..., 0.0980, 0.0980, 0.0863],
         [0.3882, 0.4000, 0.3569,  ..., 0.1176, 0.1176, 0.

In [10]:
img_tensor.shape

torch.Size([3, 512, 512])

In [11]:
out = model(img_tensor.unsqueeze(0))

In [12]:
out

tensor([[-1.8431, -0.4925, -0.8997,  0.1917,  4.3947, -1.0505,  3.0833, -1.7417,
         -1.4016]], grad_fn=<AddmmBackward0>)

In [13]:
pred, idx = torch.max(out, 1)

In [14]:
pred

tensor([4.3947], grad_fn=<MaxBackward0>)

In [15]:
classes[idx]

'dry'

In [16]:
def predict(model, img, tr, classes):
    img_tensor = tr(img)
    out = model(img_tensor.unsqueeze(0))
    pred, idx = torch.max(out, 1)
    return classes[idx]

In [19]:
img = Image.open('./datasets/dry/dry_skin_1.jpg').convert("RGB")
res = predict(model, img, tr, classes)
res

'oily'

In [22]:
img = Image.open('./datasets/acne-scars/acne_scar_1.jpg').convert("RGB")
res = predict(model, img, tr, classes)
res

'dry'